In [12]:
import tensorflow as tf
from tensorflow import keras
import pickle
import numpy as np
import pandas as pd

## 加载CIFAR-10数据集

In [ ]:
train_data = {b'data':[], b'labels':[]} #两个items都是list形式
# 5*10000的训练数据和1*10000的测试数据，数据为dict形式，train_data[b'data']为10000 * 3072的numpy向量
# 3072个数字表示图片特征，前1024个表示红色通道，中间1024表示绿色通道，最后1024表示蓝色通道
# train[b'labels']为长度为10000的list，每一个list数字对应以上上3072维的一个特征

(X_train_full,y_train_full),(X_test,y_test) = keras.datasets.cifar10.load_data()
#10个类别，60000个32×32像素图像，50000个用于训练，10000个用于测试

X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

#归一化输入特征
X_train = X_train/255.0
X_test = X_test/255.0
X_valid = X_valid/255.0
X_test = X_test/255.0

# 加载训练数据
for i in range(5):
    with open("C:/Users/29811/Desktop/cifar10/data/cifar-10-batches-py/data_batch_" + str(i + 1), mode='rb') as file:
        data = pickle.load(file, encoding='bytes')
        train_data[b'data'] += list(data[b'data'])
        train_data[b'labels'] += data[b'labels']

# 加载测试数据
with open("C:/Users/29811/Desktop/cifar10/data/cifar-10-batches-py/test_batch", mode='rb') as file:
    test_data = pickle.load(file, encoding='bytes')

# 定义一些变量
NUM_LABLES = 10 # 分类结果为10类
FC_SIZE = 384   # 全连接隐藏层节点个数
BATCH_SIZE = 32 # 每次训练batch数
lamda = 0.004   # 正则化系数，这里还未用正则化处理
sess = tf.InteractiveSession()


## 构建预处理函数

In [4]:
def _get_file_path(filename=''):
    """ 
    Return the full path of a data-file for the data-set.
    If filename=="" then return the directory of the files.
    """
    return os.path.join(data_path,'cifar-10-batches-py/',filename)

In [5]:
def _unpickle(filename):
    """ 
    Unpickle the given file and return the data.
    Note that the appropriate dir-name is prepended
    the filename.
    """
    file_path = _get_file_path(filename)
    print("Loading data: " + file_path)
    with open(file_path, mode='rb') as file:
        if python_version == "2":
            data = pickle.load(file)
        else:
            data = pickle.load(file, encoding="bytes")
    return data

def _convert_images(raw):
    """
    Convert images from unpickled data (10000, 3072)
    to a 4-dim array
    
    Args:
        raw: unpackled data from cifar10, eg: (10000,3072)
    return:
        a 4-dim array: (img_num, height, width, channel)
    """
    num_channels = 3
    img_size = 32
    raw_float = np.array(raw, dtype=float)/255.0
    images = raw_float.reshape([-1,num_channels,img_size,img_size])
    images = images.transpose([0, 2, 3, 1])
    return images

In [6]:
def _load_data(filename):
    """
    Load a pickled data-file from the CIFAR-10 data set
    and return the converted images (see above) and the 
    class-number for each image.
    """
    data = _unpickle(filename)
    if python_version == "2":
        raw_images = data['data'] # delete 'b' when using python2
        labels = np.array(data['labels']) # delete 'b' when using python2
    else:
        raw_images = data[b'data']
        labels = np.array(data[b'labels'])  
    images = _convert_images(raw_images)
    return images, labels

def load_label_names():
    """
    Load the names for the classes in the CIFAR-10 data set.
    Returns a list with the names. 
    Example: names[3] is the name associated with class-number 3.
    """
    raw = _unpickle("batches.meta")
    if python_version == "2":
        label_names = [x.decode('utf-8') for x in raw['label_names']]
    else:
        label_names = raw[b'label_names']
    return label_names

def _one_hot_encoded(class_numbers, num_classes=None):
    """
    Generate the One-Hot encoded class-labels from an array of integers.

    For example, if class_number=2 and num_classes=4 then
    the one-hot encoded label is the float array: [0. 0. 1. 0.]

    Args:
        class_numbers: array of integers with class-numbers.
        num_classes: number of classes. If None then use max(cls)-1.
    Return:
        2-dim array of shape: [len(cls), num_classes]
    """
    if num_classes is None:
        num_classes = np.max(class_numbers)+1
        
    return np.eye(num_classes, dtype=float)[class_numbers] 

In [7]:
def load_training_data():
    """
    Load all the training-data for the CIFAR-10 data-set.
    The data-set is split into 5 data-files which are merged here.

    Returns:
        images: training images
        labels: label of training images
        one_hot_labels: one-hot labels.
    """ 
    num_files_train = 5
    images_per_file = 10000
    num_classes = 10
    img_size = 32
    num_channels = 3
    num_images_train = num_files_train*images_per_file
    
    # 32bit的Python使用内存超过2G之后,此处会报MemoryError(最好用64位)
    images = np.zeros(shape=[num_images_train, img_size, img_size, num_channels], dtype=float)
    labels = np.zeros(shape=[num_images_train], dtype=int)
    
    begin = 0
    for i in range(num_files_train):
        images_batch, labels_batch = _load_data(filename="data_batch_"+str(i+1)) # _load_data2 in python2
        num_images = len(images_batch)
        end = begin + num_images
        images[begin:end,:] = images_batch
        labels[begin:end] = labels_batch
        begin = end
    one_hot_labels = _one_hot_encoded(class_numbers=labels,num_classes=num_classes)
    return images, labels, one_hot_labels

In [8]:
def load_test_data():
    """
    Load all the test-data for the CIFAR-10 data-set.
    Returns:
    the images, class-numbers and one-hot encoded class-labels.
    """
    num_classes = 10
    images, labels = _load_data(filename="test_batch") # _load_data2 in python2
    return images, labels, _one_hot_encoded(class_numbers=labels, num_classes=num_classes)

In [9]:
def split_train_data(images_train, one_hot_labels_train, ratio = 0.1, shuffle = False):
    """
    split valid data from train data with specified ratio.
    
    Arguments:
        images_train: train data (50000, 32, 32, 3).
        one_hot_labels_train: one-hot labels of train data (50000, 10). 
        ratio: valid data ratio.
        shuffle: shuffle or not.  
    Return:
        images_train: splitted train data.
        one_hot_labels_train: train data labels.
        images_valid: valid data
        one_hot_labels_valid: valid data labels
    """
    num_train = images_train.shape[0]
    num_valid = int(np.math.floor(num_train * ratio))

    if shuffle:
        permutation = list(np.random.permutation(num_train))
        images_train = images_train[permutation, ]
        one_hot_labels_train = one_hot_labels_train[permutation, ]

    images_valid = images_train[-num_valid:, ]
    one_hot_labels_valid = one_hot_labels_train[-num_valid:, ]
    images_train = images_train[0:-num_valid, ]
    one_hot_labels_train = one_hot_labels_train[0:-num_valid, ]
    return images_train, one_hot_labels_train, images_valid, one_hot_labels_valid

In [10]:
def create_mini_batches(X, Y, mini_batch_size = 128, shuffle=False):
    """
    Create a list of minibatches from the training images.
    
    Arguments:
        X: numpy.ndarry images shaped (num_images, height, width, channels).
           for example: (50000, 32, 32, 3).
        Y: one-hot labels of images shaped (num_images, num_classes).
           for example: (50000,10) 
        mini_batch_size: Mini-batch size .
        shuffle: Shuffling the images or not.
    Return:
        mini_batches_X: a list of all mini-batches images, each element in
                        it is an numpy.ndarray containing one batch of images.
        mini_batches_Y: a list of all mini-batches one-hot labels, 
                        each element in it is an one-hot label.
    """
    m = X.shape[0]
    mini_batches_X = []
    mini_batches_Y = []
    
    if shuffle:
        permutation = list(np.random.permutation(m))
        X = X[permutation, ]
        Y = Y[permutation, ]
        
    num_complete_minibathes = int(np.math.floor(m/mini_batch_size))

    for k in range(0, num_complete_minibathes):
        mini_batch_X = X[k*mini_batch_size:(k+1)*mini_batch_size, ]
        mini_batch_Y = Y[k*mini_batch_size:(k+1)*mini_batch_size, ]
        
        mini_batches_X.append(mini_batch_X)
        mini_batches_Y.append(mini_batch_Y)
        
    if m % mini_batch_size != 0:
        mini_batch_X = X[num_complete_minibathes*mini_batch_size:, ]
        mini_batch_Y = Y[num_complete_minibathes*mini_batch_size:, ]
        mini_batches_X.append(mini_batch_X)
        mini_batches_Y.append(mini_batch_Y)

    return mini_batches_X, mini_batches_Y

## 构建5层CNN模型

In [11]:
def max_pool(feature_map):
    return tf.nn.max_pool(feature_map, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

def conv_relu(feature_map, weight, bias):
    conv = tf.nn.conv2d(feature_map, weight, strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv + bias)

def conv_pool_relu(feature_map, weight, bias):
    conv = tf.nn.conv2d(feature_map, weight, strides=[1, 1, 1, 1], padding='SAME')
    pool = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    return tf.nn.relu(pool + bias)

def fc_relu(fc_input, weight, bias):
    fc = tf.matmul(fc_input, weight) 
    return tf.nn.relu(fc + bias)

TabError: ignored

In [ ]:
def cifar10_5layers(input_image, keep_prob, init_method=tf.truncated_normal_initializer(stddev=1e-2)):
    """ 
    model definition with 5 layers.
    Args:
        input_image: input image tensor.
        init_method: initialization method. 
                     The default is tf.truncated_normal_initializer(1e-2)
    Return:
        model: computation graph of the defined model.
    """
    with tf.variable_scope("conv1"):
        W1 = tf.get_variable(name="W1", shape=[5,5,3,32], dtype=tf.float32, 
                             initializer=init_method)
        b1 = tf.get_variable(name="b1", shape=[32], dtype=tf.float32, 
                             initializer=tf.constant_initializer(0.01))
        conv1 = conv_pool_relu(input_image, W1, b1)
    with tf.variable_scope("conv2"):
        W2 = tf.get_variable(name="W2", shape=[5,5,32,64], dtype=tf.float32, 
                             initializer=init_method)
        b2 = tf.get_variable(name="b2", shape=[64], dtype=tf.float32, 
                             initializer=tf.constant_initializer(0.01))
        conv2 = conv_pool_relu(conv1, W2, b2)
	      conv2 = tf.nn.dropout(conv2, keep_prob)
    with tf.variable_scope("conv3"):
        W3 = tf.get_variable(name="W3", shape=[5,5,64,128], dtype=tf.float32, 
                             initializer=init_method)
        b3 = tf.get_variable(name="b3", shape=[128], dtype=tf.float32, 
                             initializer=tf.constant_initializer(0.01))
        conv3 = conv_pool_relu(conv2, W3, b3)
	      conv3 = tf.nn.dropout(conv3, keep_prob)
    with tf.variable_scope("fc1"):
        W4 = tf.get_variable(name="W4", shape=[4*4*128,256], dtype=tf.float32, 
                             initializer=init_method)
        b4 = tf.get_variable(name="b4", shape=[256], dtype=tf.float32, 
                             initializer=tf.constant_initializer(0.01))
        conv3_flat = tf.reshape(conv3, [-1, 4*4*128])
        fc1 = fc_relu(conv3_flat, W4, b4)
	      fc1 = tf.nn.dropout(fc1, keep_prob)
    with tf.variable_scope("output"):
        W5 = tf.get_variable(name="W5", shape=[256,10], dtype=tf.float32, 
                             initializer=init_method)
        b5 = tf.get_variable(name="b5", shape=[10], dtype=tf.float32, 
                             initializer=tf.constant_initializer(0.01))
	      y_logit = tf.matmul(fc1, W5) + b5
    return y_logit, tf.nn.softmax(y_logit, name="softmax")

## 训练模型

In [ ]:
init_methods = {"Gaussian": tf.truncated_normal_initializer(stddev=1e-2), 
                "Xavier": tf.contrib.layers.xavier_initializer_conv2d(),
                "He": tf.contrib.layers.variance_scaling_initializer()}

def parse_args():
	"""
	parse input arguments.
	"""
	parse = argparse.ArgumentParser(description="CIFAR-10 training") 
	parse.add_argument("--model", dest="model_name", 
					   help="model name: 'cifar10-5layers' or 'cifar10-8layers'",
					   default="cifar10-5layers")
	parse.add_argument("--init", dest="init_method",
					   help="initialization method for weights, 'Gaussian', 'Xavier' or 'He'",
					   default="Gaussian")
	args = parse.parse_args() # 获取所有的参数
	return args

def main():
    """
    # 下载并解压数据集(已下载可忽略)
    data_url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
    data_path = "./data/"
    download.maybe_download_and_extract(data_url,data_path)
    """
    args = parse_args()
    # 导入数据集并显示数据集信息
    class_names= load_label_names()
    images_train, _, labels_train = load_training_data()
    images_test, _, labels_test = load_test_data()
    images_train, labels_train, images_valid, labels_valid = \
        split_train_data(images_train, labels_train, shuffle=True)

    print("classes names:", class_names)
    print("shape of training images:", images_train.shape)
    print("shape of training labels (one-hot):", labels_train.shape)
    print("shape of valid images:", images_valid.shape)
    print("shape of valid labels (one-hot):", labels_valid.shape)
    print("shape of test images:", images_test.shape)
    print("shape of testing labels (one-hot):", labels_test.shape)

    # 将数据集分成mini-batches.
    images_train_batches, labels_train_batches = create_mini_batches(images_train, \
                                                                    labels_train, \
                                                                    shuffle=True)
    print("shape of one batch training images:", images_train_batches[0].shape)
    print("shape of one batch training labels:", labels_train_batches[0].shape)
    print("shape of last batch training images:", images_train_batches[-1].shape)
    print("shape of last batch training labels:", labels_train_batches[-1].shape)

    img_size = images_train.shape[1]
    num_channels = images_train.shape[-1]
    num_classes = len(class_names)
    batch_size = images_train_batches[0].shape[0]
    num_batches = len(images_train_batches)

    # 创建模型
    X = tf.placeholder(tf.float32, [None, img_size, img_size, num_channels])
    Y_ = tf.placeholder(tf.float32, [None, num_classes])
    keep_prob = tf.placeholder(tf.float32)

    init_method = init_methods[args.init_method]
    if args.model_name == "cifar10-5layers":
        logit, Y = cifar10_5layers(X, keep_prob, init_method)
    elif args.model_name == "cifar10-8layers":
        logit, Y = cifar10_8layers(X, keep_prob, init_method)

    # 交叉熵损失和准确率
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logit, labels=Y_)
    cross_entropy = tf.reduce_mean(cross_entropy)
    correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    # 训练过程
    tr_step = []; tr_acc = []; tr_loss = []
    va_step = []; va_acc = []; va_loss = []
    train_steps =50000 
    lr = 0.0001
    train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy)
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    with tf.Session() as sess:
        sess.run(init)
        for i in range(train_steps+1):
            # 获取一个mini batch数据
            j = i%num_batches # j用于记录第几个mini batch
            batch_X = images_train_batches[j]
            batch_Y = labels_train_batches[j]
            if j == num_batches-1: # 遍历一遍训练集（1个epoch）
                images_train_batches, labels_train_batches = create_mini_batches(images_train, \
                                                                                labels_train, \
                                                                                shuffle=True)

            # 训练一次
            sess.run(fetches=train_step, feed_dict={X: batch_X, Y_: batch_Y, keep_prob: 0.5})
            
            # 每100次打印并记录一组训练结果
            if i % 100 == 0:
                train_accuracy, train_loss = sess.run([accuracy, cross_entropy], \
                                                      feed_dict={X: batch_X, Y_: batch_Y, keep_prob: 1})
                print("steps =", i, "train loss =", train_loss, " train accuracy =", train_accuracy)
                tr_step.append(i)
                tr_acc.append(train_accuracy)
                tr_loss.append(train_loss)

            # 每500次打印并记录一次测试结果（验证集）
            if i % 500 == 0:
                valid_accuracy, valid_loss = sess.run([accuracy, cross_entropy], \
                                                      feed_dict={X: images_valid, Y_: labels_valid, keep_prob: 1})
                va_step.append(i)
                va_acc.append(valid_accuracy)
                va_loss.append(valid_loss)
                print("steps =", i, "validation loss =", valid_loss, " validation accuracy =", valid_accuracy)
            
            # 每10000次保存一次训练模型到本地
            if i % 10000 == 0 and i > 0:
                model_name = args.model_name + "_" + args.init_method
                model_name = os.path.join("./models", model_name)
                saver.save(sess, model_name, global_step=i)

    # 保存训练日志到本地  
    train_log = "train_log_" + args.model_name + "_" + args.init_method + ".txt" 
    train_log = os.path.join("./results", train_log)
    with open(train_log, "w") as f:
        f.write("steps\t" + "accuracy\t" + "loss\n")
        for i in range(len(tr_step)):
            row_data = str(tr_step[i]) + "\t" + str(round(tr_acc[i],3)) + "\t" + str(round(tr_loss[i],3)) + "\n"
            f.write(row_data)

    valid_log = "valid_log_" + args.model_name + "_" + args.init_method + ".txt" 
    valid_log = os.path.join("./results", valid_log)            
    with open(valid_log, "w") as f:
        f.write("steps\t" + "accuracy\t" + "loss\n")
        for i in range(len(va_step)):
            row_data = str(va_step[i]) + "\t" + str(round(va_acc[i],3)) + "\t" + str(round(va_loss[i],3)) + "\n"
            f.write(row_data)

if __name__ == "__main__":
    main()

## 测试模型

In [ ]:
def parse_args():
    	"""
	parse input arguments.
	"""
	parse = argparse.ArgumentParser(description="CIFAR-10 test") 
	parse.add_argument("--model", dest="model_name", 
					   help="model name: 'cifar10-5layers' or 'cifar10-8layers'",
					   default="cifar10-5layers")
	parse.add_argument("--path", dest="model_path",
					   help="trained model file path: ***.data***",
					   default="models/cifar10-5layers_Gaussian-10000.data-00000-of-00001")
	args = parse.parse_args()
	return args


def main():
    args = parse_args()
    model_name = args.model_name
    model_path = args.model_path.split(".")[0]
    # 加载测试数据集
    images_test, _, labels_test = load_test_data()
    print("images_test.shape = ", images_test.shape)

    # 构建模型（模型也可以直接从.meta文件中恢复）
    X = tf.placeholder(tf.float32, [None, 32, 32, 3])
    Y_ = tf.placeholder(tf.float32, [None, 10])
    keep_prob = tf.placeholder(tf.float32)
    if model_name == "cifar10-5layers":
        _, Y = cifar10_5layers(X, keep_prob)
    elif model_name == "cifar10-8layers":
        _, Y = cifar10_8layers(X, keep_prob)
    else:
        print("wrong model name!")
        print("model name: 'cifar10-5layers' or 'cifar10-8layers'")
        raise KeyError

    correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    saver = tf.train.Saver()
    with tf.Session() as sess:
        # 恢复模型权重
        saver.restore(sess, model_path)
        # 测试，注意测试阶段keep_prob=1
        print("test accuracy = ", sess.run(accuracy, feed_dict={X: images_test, Y_: labels_test, keep_prob: 1}))

if __name__ == "__main__":
    main()